In [31]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# SET VALIDATE = TRUE TO COMPUTE VALIDATION. 
# AND SET VALIDATE = FALSE TO MAKE AN SUBMISSION TO LB
VALIDATE = True

# USE 1 TO VALIDATE WITH LAST MONTH OF TRAIN, 2 FOR SECOND TO LAST ETC ETC
# NEW TRAIN DATA IS EVERYTHING BEFORE THIS
VAL_MONTH = 1

In [32]:
# LOAD TRAIN DATA
train = pd.read_csv('./data/train.csv')
train["first_day_of_month"] = pd.to_datetime(train["first_day_of_month"])
train = train.sort_values(['cfips','first_day_of_month']).reset_index(drop=True)


In [33]:
# DEFINE VALIDATION AND TRAIN MONTHS
trn_months = train.first_day_of_month.values[-39:-1*VAL_MONTH]
val_months = train.first_day_of_month.values[-1*VAL_MONTH:]
if VAL_MONTH!=1: 
    val_months = train.first_day_of_month.values[-1*VAL_MONTH:-1*VAL_MONTH+1]


In [35]:
# CREATE NEW TRAIN AND NEW TEST DATA IF WE ARE VALIDATING
if VALIDATE:
    test = train.loc[train.first_day_of_month.isin(val_months)]
    train = train.loc[train.first_day_of_month.isin(trn_months)]
print( train.shape )
train.head()

(119130, 7)


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [37]:
test = test.sort_values(['cfips','first_day_of_month']).reset_index(drop=True)
print(test.shape)
test.head()

(3135, 7)


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2022-10-01,1001,Autauga County,Alabama,2022-10-01,3.463856,1472
1,1003_2022-10-01,1003,Baldwin County,Alabama,2022-10-01,8.359798,14320
2,1005_2022-10-01,1005,Barbour County,Alabama,2022-10-01,1.232074,244
3,1007_2022-10-01,1007,Bibb County,Alabama,2022-10-01,1.287240,229
4,1009_2022-10-01,1009,Blount County,Alabama,2022-10-01,1.831783,813


In [39]:
# COUNT MONTHS OF NEW TRAIN AND NEW TEST
TRAIN_SZ = len(train)//3135
TEST_SZ = len(test)//3135
print('New train data has',TRAIN_SZ,'months and new test data has', TEST_SZ,'months')

New train data has 38 months and new test data has 1 months


In [40]:
DISPLAY = 8
# ERROR THRESHOLD AS PERCENTAGE = THRESHOLD / 78
THRESHOLD = 8 # for linear regression
ACTIVE_THRESHOLD = 9_000

IDS = train.cfips.unique()
x_train = np.arange(TRAIN_SZ).reshape((-1,1))
x_test = np.arange(TRAIN_SZ-1,TRAIN_SZ+TEST_SZ).reshape((-1,1))

linear_preds = np.zeros((len(IDS),TEST_SZ))
last_preds = np.zeros((len(IDS),TEST_SZ))
seasonal_preds = np.zeros((len(IDS),TEST_SZ))
sn_trend = 0
lin_trend = 0

In [78]:
i = 0 
c = IDS[1]

df = train.loc[train.cfips==c]
last = df.microbusiness_density.values[-1]
active = df.active.values[-1]
last_preds[i,] = [last]*TEST_SZ
# df


In [79]:
WIDTH1 = 5; WIDTH2 = 7; WIDTH3 = 7
# RECENT
x0a = df.microbusiness_density.values[-1-WIDTH1:-1]

In [80]:
model = LinearRegression()
model.fit(x_train,df.microbusiness_density)

LinearRegression()

In [94]:
p = model.predict(x_train)

# plt.plot(x_train,df.microbusiness_density)
# plt.plot(x_train,p)